### Cache verification results to verif_result.json

In [1]:
import json
import data
import evaluation

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/omkar/.config/sagemaker/config.yaml


In [12]:
RESULT_FILE = "nagini_examples/tree_finetune/verif_result.json"
KEY = "tree"
dataset = data.Data("config.json", "nagini_examples/tree_finetune")
ev = evaluation.Evaluation(dataset)
examples = ev.data.get_list_of_examples(KEY)
verif_result:dict[str,str] = {}

# with open(RESULT_FILE, "rb") as f:
#     verif_result = json.load(f)
# print(verif_result)

for example in examples:
    program_snippet = ev.data.get_example(KEY, example, "unverified")
    result = ev.verify_program_snippet(KEY, program_snippet, method_name=example)
    verif_result[example] = str(result)

with open(RESULT_FILE, "w") as f:
    json.dump(verif_result, f, indent=4)

{'val_head': 'Verification failed: Assignment might fail. There might be insufficient permission to access node.key. at line 3.4', 'height': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.16', 'count': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.12', 'sum': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.19', 'insert': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access node.key. at line 3.7', 'contains': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access node.key. at line 5.7', 'inorder': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.4', 'min': 'Verification failed: Conditional statement might fail. There 

In [13]:
verif_result

{'val_head': 'Verification failed: Assignment might fail. There might be insufficient permission to access node.key. at line 3.4',
 'height': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.16',
 'count': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.12',
 'sum': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.19',
 'insert': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access node.key. at line 3.7',
 'contains': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access node.key. at line 5.7',
 'inorder': 'Verification failed: Method call might fail. There might be insufficient permission to access node.left. at line 5.4',
 'min': 'Verification failed: Conditional statement might fail.

In [ ]:
import json
with open("nagini_examples/verif_result_sorting.json", "w") as f:
    json.dump(verif_result, f, indent=4)

Script to create unverified programs given the verified program and write them to the dataset.

In [11]:
import data
import ast
import ast_util

KEY = "tree"
DATA_ROOT = "nagini_examples/tree_finetune"
dataset = data.Data("config.json", DATA_ROOT)
examples = dataset.get_list_of_examples(KEY)
for example in examples:
    program_snippet = dataset.get_example(KEY, example, "verified")
    program_ast = ast.parse(program_snippet, type_comments=True)
    unverifed_ast = ast_util.remove_all_annotations(program_ast)
    unverified = ast.unparse(unverifed_ast)
    dataset.add_unverified(example, unverified, KEY)